In [1]:
#import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, time
from DynamicTimeModel import DynTM
from Helper import *
from queryAlgorithm import earliestArrival_path, pareto_paths

### Import data

In [2]:
timetable = pd.read_csv('../dataPreprocessing/data/preprocessed_timetable.csv')
#timetable = timetable.sort_values(['nextStop_arrival_time'])
timetable.head()

,TripNb,line_id,vehicle_type,currentStop,nextStop,departure_time,nextStop_arrival_time,nextStop_deaprture_time
0,1,10,1,72,73,03:36:01,03:53:27,03:53:42
1,1,10,1,73,74,03:53:42,03:55:36,03:55:52
2,1,10,1,74,75,03:55:52,03:57:00,03:57:17
3,1,10,1,75,76,03:57:17,03:58:42,03:58:58
4,1,10,1,76,77,03:58:58,03:59:52,04:00:06


In [3]:
#convert str to time object
timetable.departure_time  = timetable.departure_time.apply(str_time_to_int)
timetable.nextStop_arrival_time = timetable.nextStop_arrival_time.apply(str_time_to_int)
timetable.nextStop_deaprture_time = timetable.nextStop_deaprture_time.apply(str_time_to_int)

In [4]:
Dmatrix = pd.read_csv('../dataPreprocessing/data/Dmatrix.csv')
Dmatrix = Dmatrix.set_index('station_id')

In [5]:
stations_paths = pd.read_csv('../dataPreprocessing/data/stations_paths.csv')
stations_coord = pd.read_csv('../dataPreprocessing/data/station_coord.csv')
paths = pd.read_csv('../dataPreprocessing/data/paths.csv')

In [6]:
stations = pd.concat([timetable.currentStop,timetable.nextStop]).reset_index(drop=True).unique()
stations_paths = stations_paths[stations_paths.station_id.isin(stations)]
stations_paths = stations_paths[['station_id', 'english_name', 'french_name', 'arabic_name','latitude','longitude']].drop_duplicates()

In [7]:
stations_coord = stations_coord[stations_coord.station_id.isin(stations)]


## Build DTM Graph Structure

In [8]:
#timetable data structre
timetable = (stations ,timetable.values)

In [9]:
timetable[0].shape

(290,)

In [10]:
timetable[1].shape

(27967, 8)

In [11]:
#intiate graph
max_walk_between_stations = 300 #300meters
walk_speed  = 1.4  #1.4 meter per second
DTM = DynTM(max_walk_between_stations, stations_coord, walk_speed)

In [12]:
from time import time
#generate graph and add unrestricted path
start = time()
DTM.generateTimetableDynamicModel(timetable)
DTM.add_unresticted_paths(Dmatrix)
stop = time()
print(stop-start)

0.8255414962768555


In [13]:
DTM.dtm.number_of_nodes()

30205

In [14]:
DTM.dtm.number_of_edges()

84441

In [15]:
start = time()
update = 0
delete = []
for node in DTM.dtm.nodes:
    if 'switch' not in node and (1 == node[-2] or 2 == node[-2]) and node[1] == 10 :
        delete.append(node)

In [16]:
delete

[('node', 10, 72, 1, 12961),
 ('node', 10, 73, 1, 14022),
 ('node', 10, 74, 1, 14152),
 ('node', 10, 75, 1, 14237),
 ('node', 10, 76, 1, 14338),
 ('node', 10, 77, 1, 14406),
 ('node', 10, 78, 1, 14493),
 ('node', 10, 79, 1, 14593),
 ('node', 10, 80, 1, 14697),
 ('node', 10, 81, 1, 14778),
 ('node', 10, 82, 1, 14858),
 ('node', 10, 83, 1, 14930),
 ('node', 10, 84, 1, 15056),
 ('node', 10, 85, 1, 15165),
 ('node', 10, 86, 1, 15296),
 ('node', 10, 87, 1, 15426),
 ('node', 10, 88, 1, 15525),
 ('node', 10, 72, 2, 15841),
 ('node', 10, 73, 2, 16902),
 ('node', 10, 74, 2, 17032),
 ('node', 10, 75, 2, 17117),
 ('node', 10, 76, 2, 17218),
 ('node', 10, 77, 2, 17286),
 ('node', 10, 78, 2, 17373),
 ('node', 10, 79, 2, 17473),
 ('node', 10, 80, 2, 17577),
 ('node', 10, 81, 2, 17658),
 ('node', 10, 82, 2, 17738),
 ('node', 10, 83, 2, 17810),
 ('node', 10, 84, 2, 17936),
 ('node', 10, 85, 2, 18045),
 ('node', 10, 86, 2, 18176),
 ('node', 10, 87, 2, 18306),
 ('node', 10, 88, 2, 18405),
 ('node', 10, 

In [17]:
t_derive = timetable[1][:34]

In [18]:
moy = np.zeros(1000)
for i in range(1000):
    start = time()
    #delete departure nodes
    for node in delete:
        DTM.dtm.remove_node(node)
    #add departure nodes
    DTM.update_departureEvents(t_derive)
    stop = time()
    moy[i] = stop-start

In [19]:
moy.mean()

0.0004194412231445312

## DTM-QH algorithm to solve EA Problem

In [20]:
from time import time as timo

curr_time = datetime.now().time().replace(microsecond=0)
curr_time = str(curr_time.hour)+':'+str(curr_time.minute)+':'+str(curr_time.second)
curr_time = str_time_to_int(curr_time)
curr_time = str_time_to_int('10:10:00')

#source = ('source_node', 36.87007096206944, 10.34382253848957)   #sidi bou said
#source = ('source_node', 36.87970216988326, 10.327309234021802)   #marsa ville
#target = ('target_node', 36.8226374125104, 10.30882462463282) #la goulette
#source = ('source_node', 36.83906834458209, 10.316478568193) #le kram
#target = ('target_node',36.8066962256805, 10.18126936246436) #siege IT transtu 
source = ('source_node', 36.806678, 10.181269)   #Tunis marine
#target = ('target_node', 36.829968, 10.150268)  #manar1
#target = ('target_node', 36.724688, 10.205779)  #el mourouj 3
#target = ('target_node',36.85340441371543, 10.196437250933252)  #ariana
#target = ('target_node', 36.81792285359155, 10.080213439106986)  #manouba
#target = ('target_node', 36.75537489487085, 10.22252052118309 ) #Ben Arouss
#target = ('target_node', 36.83848676642428, 10.183395901376352) #Manzeh 1 cité sportive
#target = ('target_node', 36.80936808780992, 10.133522078274913) #Musée bardo
target = ('target_node', 36.796247434333004, 10.132042159557537) #ezzouhour
#target = ('target_node', 36.79086785239071, 10.092124602977709) #cité el mechtel
#target = ('target_node', 36.841494756981, 10.132237773927486) #Omrane sup
#target = ('target_node', 36.79037011799913, 10.107341203160626) #municipalité hrairia (ezzahrouni)
#target = ('target_node', 36.788855425031464, 10.177730508869596) #beb alioua
#target = ('target_node', 36.839583107126735, 10.117761184660045) #el intilaka

In [21]:
#add source node
coords =  source[1:]
DTM.add_sourceNode(coords, 2000)

In [22]:
#add target node
coords =  target[1:]
DTM.add_targetNode(coords, 2000)

In [23]:
start = timo()
path, transitions, walk = pareto_paths(DTM.dtm, source, target, curr_time, walk_panalisation=True, transfer_time=150)
stop = timo()
print(stop - start)

1.8308074474334717


In [24]:
walk

[650]

In [25]:
transitions

[1]

In [26]:
path

[['switch_source',
  ('switch', 161.0),
  ('node', 35, 161, 4, 37383),
  ('node', 35, 162, 4, 37969),
  ('node', 35, 159, 4, 38043),
  ('node', 35, 152, 4, 38137),
  ('node', 35, 23, 4, 38291),
  ('node', 35, 192, 4, 38470),
  ('node', 35, 194, 4, 38586),
  ('node', 35, 195, 4, 38620),
  ('node', 35, 196, 4, 38688),
  ('switch', 197),
  'switch_target']]

In [27]:
'''
print('---------------------Pareto--Set--Of--Optimal--Paths--Solution------------------------', end='\n\n')
for i, solution in enumerate(path) : 
    print('*Path N°',i+1, ' : ', end='\n\n')
    print(path_interpretor(solution, stations_paths, paths, language='french'))
    print('-------------------------------------------------------------------------------------', end='\n\n')
'''

"\nprint('---------------------Pareto--Set--Of--Optimal--Paths--Solution------------------------', end='\n\n')\nfor i, solution in enumerate(path) : \n    print('*Path N°',i+1, ' : ', end='\n\n')\n    print(path_interpretor(solution, stations_paths, paths, language='french'))\n    print('-------------------------------------------------------------------------------------', end='\n\n')\n"

In [28]:
start = timo()
path, dist, transistions, cc = earliestArrival_path(DTM.dtm, source, target, curr_time, walk_panalisation=True, transfer_time=150)
stop = timo()
print(stop - start)

0.07592582702636719


In [29]:
DTM.dtm[('switch', 89.0)]

AtlasView({('node', 11, 89, 1, 15331): {'weight': 100000}, ('node', 11, 89, 2, 18031): {'weight': 100000}, ('node', 11, 89, 3, 20731): {'weight': 100000}, ('node', 11, 89, 4, 22171): {'weight': 100000}, ('node', 11, 89, 5, 23251): {'weight': 100000}, ('node', 11, 89, 6, 24331): {'weight': 100000}, ('node', 11, 89, 7, 25231): {'weight': 100000}, ('node', 11, 89, 8, 26131): {'weight': 100000}, ('node', 11, 89, 9, 27031): {'weight': 100000}, ('node', 11, 89, 10, 27931): {'weight': 100000}, ('node', 11, 89, 11, 28831): {'weight': 100000}, ('node', 11, 89, 12, 29731): {'weight': 100000}, ('node', 11, 89, 13, 30631): {'weight': 100000}, ('node', 11, 89, 14, 31531): {'weight': 100000}, ('node', 11, 89, 15, 32431): {'weight': 100000}, ('node', 11, 89, 16, 33331): {'weight': 100000}, ('node', 11, 89, 17, 34231): {'weight': 100000}, ('node', 11, 89, 18, 35131): {'weight': 100000}, ('node', 11, 89, 19, 36031): {'weight': 100000}, ('node', 11, 89, 20, 36931): {'weight': 100000}, ('node', 11, 89, 2

In [30]:
path

['switch_source',
 ('switch', 161.0),
 ('node', 35, 161, 4, 37383),
 ('node', 35, 162, 4, 37969),
 ('node', 35, 159, 4, 38043),
 ('node', 35, 152, 4, 38137),
 ('node', 35, 23, 4, 38291),
 ('node', 35, 192, 4, 38470),
 ('node', 35, 194, 4, 38586),
 ('node', 35, 195, 4, 38620),
 ('node', 35, 196, 4, 38688),
 ('switch', 197),
 'switch_target']

In [31]:
'''
print('---------------------Earliest--Arrival--Time--Path------------------------', end='\n\n')
print(path_interpretor(path, stations_paths, paths, language='french'))
'''

"\nprint('---------------------Earliest--Arrival--Time--Path------------------------', end='\n\n')\nprint(path_interpretor(path, stations_paths, paths, language='french'))\n"

In [32]:
#remove source and target nodes from graph
DTM.remove_switch('switch_source')
DTM.remove_switch('switch_target')